In [6]:
!pip install sentence_transformers

  Created wheel for sentence-transformers: filename=sentence_transformers-2.1.0-py3-none-any.whl size=120999 sha256=05ddd9fd77b8bf39e9e581322a7a0dbe6d1a1011a6e6b6049de8e278b2fc895d
  Stored in directory: c:\users\eason.chen\appdata\local\pip\cache\wheels\52\19\88\6625593382e23a926740e6fcee0f2df0a0de25766094842a28
Successfully built sentence-transformers


In [67]:
my_file = open("text.txt", "r")
content = my_file.read()
content_list = content.split(",")
my_file.close()
print(content_list)

['Bryson Research Group Computational Systems Biology Overview Teaching Suggested Projects Biography Publications Computational Systems Biology Our lab applies computational and mathematical techniques to understand biological systems that generally have clinical relevance. One area of research is modelling bacteria at the atomic', " molecular and population levels. Bacteria are relatively simple and so we can attempt to gain a mechanistic-level understanding of their working. Gaining such an understanding is clinicially important in terms of finding new approaches to tackle drug-resistant strains. A different area of our research looks at more complex systems where gaining a mechanistic understanding is not viable. In particular we are interested in neurodegenerative diseases such as Alzheimer's", " Parkinson's and Huntington's disease. We approach understanding these complex systems by integrating different types of high-throughput data and applying a variety of machine learning and 

In [68]:
from sentence_transformers import SentenceTransformer
model = SentenceTransformer('distilbert-base-nli-mean-tokens')
embeddings = model.encode(content_list)
#embeddings = embeddings.reshape(-1,1)

In [9]:
!pip install umap

  Created wheel for umap: filename=umap-0.1.1-py3-none-any.whl size=3564 sha256=3ddd3ccd4bf3cd3175c4cce49b3211725427479e5ef5cd7c29383dcc6a564f85
  Stored in directory: c:\users\eason.chen\appdata\local\pip\cache\wheels\d4\13\91\2e752dc8dab5df027854bd33d2b65e1dc5cdc107fd1133990f
Successfully built umap


In [11]:
!pip install umap-learn

  Created wheel for umap-learn: filename=umap_learn-0.5.2-py3-none-any.whl size=82696 sha256=966531d9fb9e45c9c22690a25edce060801d8502c51f8af8071c8ad7c0ac478d
  Stored in directory: c:\users\eason.chen\appdata\local\pip\cache\wheels\f2\64\75\df601da9514261c8cb0830b9515d2b94b5a51f09ddeae92b9e
  Created wheel for pynndescent: filename=pynndescent-0.5.5-py3-none-any.whl size=52587 sha256=b6d625b2cf7983a845a7256807f88b36f9af9465a41ca0655d009cbb947ed6ff
  Stored in directory: c:\users\eason.chen\appdata\local\pip\cache\wheels\e4\0e\b5\07c0c231aacb04e5d1046fe7459bb27ea79f95b5edbe88e435
Successfully built umap-learn pynndescent


In [69]:
import umap.umap_ as umap


In [73]:
umap_embeddings = umap.UMAP(n_neighbors=2, 
                            n_components=1, 
                            metric='cosine').fit_transform(embeddings)

In [72]:
!pip install hdbscan

In [74]:

import hdbscan
cluster = hdbscan.HDBSCAN(min_cluster_size=15,
                          metric='euclidean',                      
                          cluster_selection_method='eom').fit(umap_embeddings)

In [75]:
import matplotlib.pyplot as plt

# Prepare data
umap_data = umap.UMAP(n_neighbors=15, n_components=2, min_dist=0.0, metric='cosine').fit_transform(embeddings)
result = pd.DataFrame(umap_data, columns=['x', 'y'])
result['labels'] = cluster.labels_

C:\Users\eason.chen\Anaconda3\lib\site-packages\umap\umap_.py:2344: UserWarning: n_neighbors is larger than the dataset size; truncating to X.shape[0] - 1
  warn(


In [76]:
docs_df = pd.DataFrame(content_list, columns=["Doc"])
docs_df['Topic'] = cluster.labels_
docs_df['Doc_ID'] = range(len(docs_df))
docs_per_topic = docs_df.groupby(['Topic'], as_index = False).agg({'Doc': ' '.join})

In [77]:

import numpy as np
from sklearn.feature_extraction.text import CountVectorizer

def c_tf_idf(documents, m, ngram_range=(1, 1)):
    count = CountVectorizer(ngram_range=ngram_range, stop_words="english").fit(documents)
    t = count.transform(documents).toarray()
    w = t.sum(axis=1)
    tf = np.divide(t.T, w)
    sum_t = t.sum(axis=0)
    idf = np.log(np.divide(m, sum_t)).reshape(-1, 1)
    tf_idf = np.multiply(tf, idf)

    return tf_idf, count
  
tf_idf, count = c_tf_idf(docs_per_topic.Doc.values, m=len(data))

In [78]:
def extract_top_n_words_per_topic(tf_idf, count, docs_per_topic, n=20):
    words = count.get_feature_names()
    labels = list(docs_per_topic.Topic)
    tf_idf_transposed = tf_idf.T
    indices = tf_idf_transposed.argsort()[:, -n:]
    top_n_words = {label: [(words[j], tf_idf_transposed[i][j]) for j in indices[i]][::-1] for i, label in enumerate(labels)}
    return top_n_words

def extract_topic_sizes(df):
    topic_sizes = (df.groupby(['Topic'])
                     .Doc
                     .count()
                     .reset_index()
                     .rename({"Topic": "Topic", "Doc": "Size"}, axis='columns')
                     .sort_values("Size", ascending=False))
    return topic_sizes

top_n_words = extract_top_n_words_per_topic(tf_idf, count, docs_per_topic, n=20)
topic_sizes = extract_topic_sizes(docs_df); topic_sizes.head(10)

,Topic,Size
0,-1,12


In [79]:
top_n_words[-1][:10]

[('data', 0.22105663849357624),
 ('project', 0.17193515657377895),
 ('level', 0.14652345295965485),
 ('understanding', 0.14652345295965485),
 ('cell', 0.14652345295965485),
 ('systems', 0.14652345295965485),
 ('modelling', 0.14652345295965485),
 ('pathways', 0.12039518302700086),
 ('complex', 0.12039518302700086),
 ('signalling', 0.12039518302700086)]